In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import  Dense, Bidirectional, LSTM, Dropout,Input, Embedding,GRU
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.models import Model ,Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import Precision, AUC, SpecificityAtSensitivity, SensitivityAtSpecificity, binary_accuracy
from helper_functions import smi_tokenizer, mcc_metric
from rdkit import Chem
from rdkit.Chem import PandasTools
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from attention import Attention
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import class_weight

In [2]:
data = pd.read_table('F:/bbb/data/B3DB/B3DB_classification.tsv')
data.head(1)

,NO.,compound_name,IUPAC_name,SMILES,CID,logBB,BBB+/BBB-,Inchi,threshold,reference,group,comments
0,1,sulphasalazine,2-hydroxy-5-[[4-(pyridin-2-ylsulfamoyl)phenyl]...,O=C(O)c1cc(N=Nc2ccc(S(=O)(=O)Nc3ccccn3)cc2)ccc1O,5339.0,-2.69,BBB-,InChI=1S/C18H14N4O5S/c23-16-9-6-13(11-15(16)18...,NaN,R2|R2|R25|R46|,A,NaN


In [3]:
X = data["SMILES"]

target=data['BBB+/BBB-'].values.tolist()
label_encoder = LabelEncoder()
y_classification = label_encoder.fit_transform(target)

In [4]:
smile_data = [smi_tokenizer(smile) for smile in X]

In [5]:
xtrain, xtest, ytrain, ytest = train_test_split(smile_data, y_classification, test_size=0.2, random_state=42)
xtrain, xvalid, ytrain, yvalid  = train_test_split(xtrain, ytrain, test_size= 0.16, random_state=42)

In [6]:
len(y_classification)

7807

In [7]:
np.unique(y_classification, return_counts=True)

(array([0, 1], dtype=int64), array([4956, 2851], dtype=int64))

In [8]:
class_weights = class_weight.compute_class_weight(
           'balanced',
            np.unique(ytrain), 
            ytrain)

F:\anaconda\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=[0 1], y=[0 1 0 ... 0 1 0] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [9]:
class_weights

array([0.79038577, 1.36092372])

In [10]:
class_weights = {0:0.78950695,
                1: 1.36353712} 

In [11]:
text_vectorizer = TextVectorization(max_tokens=None, standardize=None, output_sequence_length=None,)

In [12]:
text_vectorizer.adapt(smile_data)
word_in_vocab = text_vectorizer.get_vocabulary()
len(word_in_vocab)

67

In [13]:
input = Input(shape = (1,), dtype = 'string')
x = text_vectorizer(input)
x = Embedding(input_dim=len(word_in_vocab),
                        output_dim=128, 
                        input_length=None,
                        )(x)
x = Bidirectional(LSTM(64, return_sequences=True,name = 'lstm1'))(x)
x = Dropout(0.2)(x)
x = Bidirectional(LSTM(64, return_sequences=True,name = 'lstm1'))(x)
x = Bidirectional(LSTM(64, return_sequences=False,name = 'lstm1'))(x)
x = Dropout(0.2)(x)

x = Dense(256, activation = 'relu')(x)
x = Dense(10, activation = 'relu')(x)
output = Dense(1, activation = "sigmoid")(x)
model = Model(inputs = input, outputs = output)


In [16]:
def save(i):
    return 'F:/bbb/model/nlp/model_'+str(i)+'.tf'

checkpoint = tf.keras.callbacks.ModelCheckpoint(save(33), 
                monitor='val_auc', verbose=1, 
                save_best_only=True, save_weights_only = False, mode='max')
callbacks_list = [checkpoint]

model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001) ,loss = BinaryCrossentropy(), metrics = [binary_accuracy, AUC(), mcc_metric 
                    ,Precision(),SensitivityAtSpecificity(specificity = 0.5), SpecificityAtSensitivity(sensitivity = 0.5)])
model.fit(x = np.array(xtrain), y = np.array(ytrain), validation_data = (np.array(xvalid), np.array(yvalid)), epochs=60, batch_size = 32, 
              callbacks = callbacks_list, class_weight = class_weights)


Epoch 1/60
164/164 [==============================] - 37s 93ms/step - loss: 0.5503 - binary_accuracy: 0.7510 - auc_2: 0.7874 - mcc_metric: nan - precision_2: 0.6817 - sensitivity_at_specificity_2: 0.8485 - specificity_at_sensitivity_2: 0.9102 - val_loss: 0.5089 - val_binary_accuracy: 0.7840 - val_auc_2: 0.8501 - val_mcc_metric: 0.5336 - val_precision_2: 0.6909 - val_sensitivity_at_specificity_2: 0.9036 - val_specificity_at_sensitivity_2: 0.9576
Epoch 2/60
164/164 [==============================] - 13s 77ms/step - loss: 0.4933 - binary_accuracy: 0.7973 - auc_2: 0.8326 - mcc_metric: 0.5687 - precision_2: 0.7291 - sensitivity_at_specificity_2: 0.8718 - specificity_at_sensitivity_2: 0.9491 - val_loss: 0.4444 - val_binary_accuracy: 0.7920 - val_auc_2: 0.8580 - val_mcc_metric: 0.5439 - val_precision_2: 0.7135 - val_sensitivity_at_specificity_2: 0.9174 - val_specificity_at_sensitivity_2: 0.9717
Epoch 3/60
164/164 [==============================] - 13s 78ms/step - loss: 0.4812 - binary_accurac

In [17]:
model.load_weights(save(33))
model.evaluate(np.array(xtest), np.array(ytest))

49/49 [==============================] - 2s 31ms/step - loss: 0.3572 - binary_accuracy: 0.8534 - auc_2: 0.9195 - mcc_metric: 0.6751 - precision_2: 0.8205 - sensitivity_at_specificity_2: 0.9786 - specificity_at_sensitivity_2: 0.9800


[0.35716164112091064,
 0.8533930778503418,
 0.9195341467857361,
 0.6750562787055969,
 0.8204632997512817,
 0.9786096215248108,
 0.9800199866294861]